In [1]:
from estnltk import Text
import csv
from bs4 import BeautifulSoup
import os
import sys
import re
from estnltk.taggers.text_segmentation.whitespace_tokens_tagger import WhiteSpaceTokensTagger
from estnltk.taggers.text_segmentation.pretokenized_text_compound_tokens_tagger import PretokenizedTextCompoundTokensTagger
from estnltk.taggers.text_segmentation.word_tagger import WordTagger
from estnltk import Layer
from estnltk.taggers.morph_analysis.morf_common import _postprocess_root

In [2]:
def read_from_tsv(path):
    texts=[]
    tokens_tagger = WhiteSpaceTokensTagger()
    if os.path.isdir(path):
        for root, dirs, files in os.walk(path):
            (head, tail) = os.path.split(root)
            if len(files) > 0:
                for file in files:
                    if not file.endswith(".tsv"):
                        continue
                    with open(os.path.join(root, file), 'r', encoding="utf-8") as fin:
                        print(file)
                        reader=csv.reader(fin, delimiter='\t')
                        words=[]
                        #Lines containing a word and its' analysis
                        word=[]
                        #Morphological analysis of the whole text.
                        morph_analysis=[]
                        raw_text=""
                        multiword_expressions = []
                        try:
                            for index, row in enumerate(reader):
                                row[0]=row[0].strip()
                                #Check if the row has correct number of elements
                                #If there are less than 6 then it is probably an adverb, abbreviation etc.
                                #But if there are more, then there's something wrond and the user has to be notified.
                                if len(row) > 6:
                                    #If the elements after the 6th one contain nothing, then we can continue.
                                    for x in row[6:]:
                                        print(row[6:])
                                        x=x.strip()
                                        if x !="":
                                            sys.stderr.write("Something is wrong with the following file: "+file+" In the following line: "+str(index+1)+"\n"+"\t".join(row)+"\n")
                                            sys.exit(1)
                                #If the first element of a row is empty then it is an alternative analysis of a word.
                                if row[0]=="" and word:
                                    word.append(row)
                                else:
                                    if len(word) != 0:
                                        words.append(word)
                                    #After appending the word into the words list let's initialize a new word.
                                    word=[row]
                            #As the loop terminates before adding the last word into the list, let's do it now
                            words.append(word)
                        except:
                            print(row[0])
                        for word in words:
                            #Iterate over the analyses and check for manual fixes.
                            #Remove all other analyses if they exist.
                            type_of_fix=""
                            for analysis in word:
                                #As it may be sometimes necessary to look at the whole line, join the elements of a row back together.
                                line="\t".join(analysis)
                                if "¤" in line:
                                    word[0][1:]=[None, None, None, None, None]
                                    word=[word[0]]

                                    type_of_fix="No_correct_analysis_available"
                                    break
                                elif analysis[1].startswith("@"):
                                    word[0][1:]=analysis[1:]
                                    word=[word[0]]
                                    word[0][1]=word[0][1].strip("@")
                                    type_of_fix="correct_analysis_provided"
                                    break
                                elif analysis[1].startswith("£"):
                                    word[0][1:]=analysis[1:]
                                    word=[word[0]]
                                    word[0][1]=word[0][1].strip("£")
                                    type_of_fix="correct_analysis_not_provided"
                                    break
                                elif re.match("#[A-Üa-ü0-9]", analysis[1]):
                                    word[0][1:]=analysis[1:]
                                    word=[word[0]]
                                    word[0][1]=word[0][1].strip("#")
                                    type_of_fix="correct_analysis_manually_added"
                                    break
                            analyses=[]
                            for a in word:
                                analysis={}
                                analysis['root']=a[1]
                                #If it is an abbreviation some fields may be missing.
                                #Sometimes there are also missing tabs in the end of a line, so the last element has to be checked.
                                if a[-1]=="Y" or a[-1]=='D' or a[-1]=='K':
                                    analysis['partofspeech']=a[-1]
                                    analysis['ending']=""
                                    analysis['form']=""
                                    analysis['clitic']=""
                                else:
                                    analysis['ending']=a[2]
                                    analysis['clitic']=a[3]
                                    analysis['partofspeech']=a[4]
                                    analysis['form']=a[5] if len(a) ==6 else ""
                                if analysis['root']!=None:
                                    analysis['root'], analysis['root_tokens'], analysis['lemma'] = _postprocess_root( analysis['root'], analysis['partofspeech'])
                                else:
                                    analysis['root_tokens']=None
                                    analysis['lemma'] =None
                                analysis['type_of_fix']=type_of_fix
                                #If not otherwize specified the normalized_text will remain the same as the word form
                                analysis['normalized_text']=word[0][0]
                                analyses.append(analysis)
                            #if len(analyses) > 1:
                            #	print (analyses)
                            word_tuple=(word[0][0], analyses)
                            morph_analysis.append(word_tuple)
                            raw_text+=word[0][0]+" "
                            if ' ' in word[0][0]:
                                multiword_expressions.append(word[0][0])
                        text = Text(raw_text)

                        tokens_layer=tokens_tagger.make_layer(text)
                        multiword_expressions = [mw.split() for mw in multiword_expressions]
                        compound_tokens_tagger = PretokenizedTextCompoundTokensTagger( multiword_units = multiword_expressions )
                        compound_tokens_layer=compound_tokens_tagger.make_layer(text, layers={'tokens':tokens_layer})
                        word_tagger=WordTagger()
                        words_layer=word_tagger.make_layer(text, layers={'compound_tokens':compound_tokens_layer, 'tokens':tokens_layer})
                        #text.tag_layer(['sentences'])
                        layer_morph=Layer(name='manual_morph',
                            text_object=text,
                            attributes=['root', 'lemma', 'root_tokens', 'ending', 'clitic', 'partofspeech', 'form'],
                            ambiguous=True)
                        layer_fix=Layer(name='type_of_fix',
                            text_object=text,
                            attributes=['type_of_fix'],
                            parent='manual_morph')

                        for ind, word in enumerate(words_layer):
                            layer_fix.add_annotation((word.start, word.end), type_of_fix=analysis['type_of_fix'])
                            for analysis in morph_analysis[ind][1]:
                                layer_morph.add_annotation((word.start, word.end), **analysis)
                        text.add_layer(layer_morph)
                        text.add_layer(layer_fix)
                        text.meta['id']=file.split(".")[0]
                        text.meta['location']=root.split(os.sep)[-1].lower()
                        texts.append(text)
    return texts

In [3]:
failid = {}

for directory in os.listdir('TSV_automaatne'):
    failid[directory] = read_from_tsv('TSV_automaatne\\'+str(directory))

al0001.tsv
al0002.tsv
al0003.tsv
al0004.tsv
al0005.tsv
al0006.tsv
al0007.tsv
al0008.tsv
al0009.tsv
al0010.tsv
al0011.tsv
al0012.tsv
al0013.tsv
al0014.tsv
al0015.tsv
al0016.tsv
al0017.tsv
al0018.tsv
al0019.tsv
al0020.tsv
al0021.tsv
al0022.tsv
al0023.tsv
al0024.tsv
al0025.tsv
al0026.tsv
al0027.tsv
al0028.tsv
al0029.tsv
al0030.tsv
al0031.tsv
al0032.tsv
al0033.tsv
al0034.tsv
al0035.tsv
al0036.tsv
al0037.tsv
al0038.tsv
al0039.tsv
al0040.tsv
al0041.tsv
al0042.tsv
al0044.tsv
al0045.tsv
al0046.tsv
al0047.tsv
al0048.tsv
al0049.tsv
al0050.tsv
al0051.tsv
al0052.tsv
al0053.tsv
al0054.tsv
al0055.tsv
al0056.tsv
al0057.tsv
al0058.tsv
al0059.tsv
al0060.tsv
al0061.tsv
al0062.tsv
al0063.tsv
al0064.tsv
al0065.tsv
al0066.tsv
al0067.tsv
al0068.tsv
al0069.tsv
al0070.tsv
al0071.tsv
al0072.tsv
al0073.tsv
al0074.tsv
al0075.tsv
al0076.tsv
al0077.tsv
al0078.tsv
al0079.tsv
al0080.tsv
al0081.tsv
al0082.tsv
al0083.tsv
al0084.tsv
al0085.tsv
al0086.tsv
al0087.tsv
al0088.tsv
al0089.tsv
al0090.tsv
al0091.tsv
al0092.tsv

KeyboardInterrupt: 

In [10]:
failid['alatskivi'][1].manual_morph.text

['<s>',
 'Keeni',
 'kogukonna',
 'kohtu',
 'kirja',
 'peale',
 'd',
 '.',
 'd',
 '.',
 'jaanuar',
 'c',
 '.',
 'nr',
 '4',
 'tuli',
 'kohtu',
 'ette',
 'Arhip',
 'Sidarov',
 'ja',
 'tunnistab',
 ',',
 'et',
 'tema',
 'olnud',
 'peremehe',
 'ja',
 'teistega',
 'Mart',
 'Uint',
 'juures',
 'müüri',
 'töös',
 ',',
 'lubi',
 'saanud',
 'enni',
 'suviste',
 'pühi',
 '4.',
 'päeva',
 'otsa',
 ',',
 'taloperemees',
 'pidanud',
 'jälle',
 'kohe',
 'kohe',
 'lubja',
 'muretsema',
 ',',
 'pole',
 'aga',
 'enni',
 'muretsenud',
 ',',
 'kui',
 'peale',
 'peetri',
 'paeva',
 ';',
 'telliskiva',
 'pole',
 'sugugi',
 'olnud',
 '.',
 '</s>',
 '<s>',
 'Tunnistaja',
 'lähnud',
 'enni',
 'jakobi',
 'paeva',
 'tööst',
 'ära',
 'ja',
 'siis',
 'pole',
 'veel',
 'telliskiva',
 'olnud',
 '.',
 '</s>',
 '<s>',
 'Keskmised',
 'põllu',
 'kivid',
 'saanud',
 'ka',
 'otsa',
 '.',
 '</s>',
 '<s>',
 'Mattei',
 'Budaschov',
 'tunnistab',
 'niisama',
 ',',
 'kui',
 'eespool',
 'Arhip',
 'Sidarov',
 '.',
 '</s>',
 '<s

In [4]:
set_of_names = set()

for key, value in failid.items():
    for text in value:
        for x in range(len(text.manual_morph.partofspeech)):
            if text.manual_morph.partofspeech[x][0] == 'H':
                set_of_names.add(text.manual_morph.text[x])
                print(text.manual_morph.text[x] + '\t' + 'propn')

Isak	propn
Karpov	propn
Bodarin	propn
Nina	propn
Anton	propn
Kusovile	propn
küla	propn
Abram	propn
Pärn	propn
Isak	propn
Karpov	propn
Bodarin	propn
kohtu	propn
kohtu	propn
Arhip	propn
Sidarov	propn
Mart	propn
Uint	propn
Mattei	propn
Budaschov	propn
Arhip	propn
Sidarov	propn
Alatskivi	propn
Kokora	propn
Kallaste	propn
küla	propn
Alatskivi	propn
Alatskivi	propn
Funcke	propn
Nikolai	propn
Semenov	propn
Mihail	propn
Artamonov	propn
Peeter	propn
Ustinov	propn
Ivan	propn
Trifanov	propn
Mikolai	propn
Grigorjev	propn
Alexei	propn
Leontjev	propn
Goruschkin	propn
Jefim	propn
Artamonov	propn
Alatskivi	propn
Alatskivi	propn
valla	propn
Joosep	propn
Roots	propn
Abram	propn
Roots	propn
Kusta	propn
Kusik	propn
Kallaste	propn
küla	propn
Alatskivi	propn
Pusi	propn
küla	propn
Alexei	propn
Leontjev	propn
Goruschkin	propn
Ivan	propn
Grigorjev	propn
Marjaschin	propn
Peeter	propn
Mikolajev	propn
Kromonov	propn
Mihaila	propn
Artamov	propn
Semen	propn
Saharov	propn
Gluharov	propn
Ivan	propn
Saveljev	propn
Glu

Mihkel	propn
Holtzmann	propn
Toomas	propn
Mihkel	propn
Holtsmann	propn
Mihkel	propn
Holtzmann	propn
Mihkel	propn
Holtzmann	propn
Grosi	propn
Mihkel	propn
Holtzmann	propn
Mihkel	propn
Holtzmann	propn
Liiso	propn
Pärni	propn
Kaarel	propn
Laht	propn
Juhanni	propn
kohtu	propn
Liso	propn
Pärnile	propn
Mart	propn
Voivot	propn
Jakob	propn
Toomassonile	propn
Joosep	propn
Leppik	propn
Annus	propn
Kuslap	propn
Jaan	propn
Kangro	propn
Kavastust	propn
kohtu	propn
Joosep	propn
Kookiga	propn
Jaan	propn
Kangro	propn
Joosep	propn
Kookile	propn
Matfei	propn
Budaschov	propn
Ivan	propn
Isakov	propn
Persitzky	propn
Persitsky	propn
Matfei	propn
Budaschov	propn
Jekim	propn
Andrejev	propn
Sadovnik	propn
Stepan	propn
Antonov	propn
Rõischov	propn
Stepan	propn
Antonov	propn
Rõischov	propn
Aleksei	propn
Stepan	propn
Antonov	propn
Rõischov	propn
Jekim	propn
Andrejev	propn
Sadovnikale	propn
kohtu	propn
Stepan	propn
Antonov	propn
Rõischov	propn
Vasili	propn
Galafejev	propn
Ulanov	propn
Kaarel	propn
Reinomägile	prop

Elias	propn
Must	propn
Johannes	propn
Kusik	propn
kohtu	propn
Jakob	propn
Kook	propn
Johannes	propn
Tuustik	propn
Johannes	propn
Tuustik	propn
Joosep	propn
Suppa	propn
Johannes	propn
Tuustik	propn
Kaarel	propn
Tikk	propn
Elias	propn
Must	propn
Johanniga	propn
Johannes	propn
Kusik	propn
Joosep	propn
Tõruvere	propn
Jakob	propn
Kook	propn
Riidma	propn
Riidmas	propn
Mihkel	propn
Indo	propn
Jaan	propn
Indo	propn
Ilja	propn
Jürgenson	propn
Abram	propn
Kook	propn
Abram	propn
Kiis	propn
Jaan	propn
Lätt	propn
Joosep	propn
Lätt	propn
August	propn
Paukam	propn
Toomas	propn
Pekk	propn
Markus	propn
Rupsi	propn
Kaarel	propn
Siim	propn
Joosep	propn
Till	propn
Joosep	propn
Tuustik	propn
Kaarel	propn
Verbo	propn
Hindrik	propn
Tilk	propn
Gross	propn
Kavastu	propn
Johan	propn
Virkhausen	propn
Mart	propn
Indo	propn
Johann	propn
Virkhausen	propn
Mart	propn
Indo	propn
Johann	propn
Virkhauseni	propn
Kavastu	propn
Jakob	propn
Pahk	propn
Alatskivi	propn
Gross	propn
Kavastu	propn
Kusta	propn
Ausenberg	propn
Ala

Kaarel	propn
Mällson	propn
Joosep	propn
Lätt	propn
Joosep	propn
Lätt	propn
Kaarel	propn
Mällson	propn
Mällson	propn
Aron	propn
Nahkur	propn
Annus	propn
Kuslap	propn
Kaarel	propn
Mällson	propn
Joosep	propn
Lätti	propn
Kaarel	propn
Mällson	propn
Joosep	propn
Lätt	propn
Aron	propn
Nahkur	propn
Mart	propn
Lummi	propn
Mart	propn
Lummi	propn
Elias	propn
Jõgi	propn
Liisa	propn
Liisa	propn
Mardiga	propn
kohtu	propn
Aron	propn
Krimm	propn
Madis	propn
Erm	propn
Tavet	propn
Kook	propn
Liisa	propn
Elias	propn
Jõgi	propn
Ann	propn
Lummi	propn
Mart	propn
Liisa	propn
Ann	propn
Lummi	propn
Mardiga	propn
kohtu	propn
Liisa	propn
Jakob	propn
Saul	propn
kohtu	propn
Elias	propn
Jõgiga	propn
Mart	propn
Lumme	propn
Ann	propn
Elias	propn
Jõgi	propn
Mart	propn
Jakob	propn
Saul	propn
Mart	propn
Liisa	propn
Mart	propn
Elias	propn
Jõgi	propn
Liisa	propn
Kuulus	propn
kohtu	propn
Miina	propn
Ränni	propn
Lisa	propn
Liisa	propn
Kuulus	propn
Miina	propn
Ränni	propn
Liisa	propn
Tartu	propn
sillakohtu	propn
Ann	propn
Lu

Kodavere	propn
Mihkel	propn
Kook	propn
Kodavere	propn
Semon	propn
Linde	propn
Kadrina	propn
valla	propn
Gustav	propn
Terras	propn
Vidrik	propn
Rossneri	propn
Semon	propn
Linde	propn
Semon	propn
Linde	propn
Mihkel	propn
Reili	propn
Kadrina	propn
Semon	propn
Linde	propn
Mihkel	propn
Reili	propn
kohtu	propn
Maeaperemees	propn
Semon	propn
Linde	propn
Mihkel	propn
Reiliga	propn
Mihkel	propn
Reili	propn
Kusta	propn
Terras	propn
Vidrik	propn
Rossner	propn
Mihkel	propn
Reili	propn
kohtu	propn
kohtu	propn
Juhan	propn
Reilile	propn
Tamme	propn
Peeter	propn
Paapu	propn
Peeter	propn
Paapule	propn
Juhan	propn
Reili	propn
Kavastu	propn
vallast	propn
kohtu	propn
Mihkel	propn
Peeter	propn
Laanel	propn
Kavastult	propn
Anton	propn
Kusmin	propn
Koslav	propn
Semon	propn
Frolov	propn
Semon	propn
Frolov	propn
Leonti	propn
Saharovile	propn
Jefim	propn
Koslov	propn
Trofim	propn
Koslov	propn
Ivan	propn
Bubnov	propn
Danila	propn
Bubnov	propn
Semon	propn
Frolov	propn
Semon	propn
Frolov	propn
Semon	propn
Frolov	p

...

Hermann	propn
Vastse-Nõo	propn
Peter	propn
Niggul	propn
Jaan	propn
Hermann	propn
kohtu	propn
Peter	propn
Niggul	propn
Kusta	propn
Beckmann	propn
Kammerist	propn
Jaan	propn
Beckmanni	propn
Raansi	propn
Kusta	propn
Peekmani	propn
Kammerin	propn
Jaan	propn
Beckmann	propn
Kusta	propn
Beckmann	propn
kohtu	propn
Jaan	propn
Hermann	propn
Peter	propn
Nigguli	propn
Peter	propn
Nigguli	propn
Tarto	propn
Kreis	propn
Kohus	propn
Seepääle	propn
Niggul	propn
Jaan	propn
Hermann	propn
Hermanni	propn
kohus	propn
Hermani	propn
Peter	propn
Niggul	propn
Johan	propn
Jaanus	propn
Jaanusse	propn
Jaanus	propn
Jaanus	propn
Kaibaja	propn
Jaanus	propn
Johan	propn
Jaanus	propn
kohtu	propn
Nigguli	propn
Nigulil	propn
Niggulile	propn
kohus	propn
Niggul	propn
Jaanus	propn
Kreis	propn
Jaanus	propn
Karl	propn
Petersell	propn
Märt	propn
Tamissar	propn
Herberi	propn
Tamissar	propn
Karl	propn
Peterselliga	propn
Tammissaar	propn
Peterselli	propn
Jaan	propn
Jänes	propn
Tamissar	propn
Karl	propn
Peterselliga	propn
Karl	prop

Juhan	propn
Juhanson	propn
Paroli	propn
Rbl	propn
Parolile	propn
Paroli	propn
Rbl	propn
Parol	propn
Rbl	propn
Parol	propn
Rbl	propn
Parol	propn
Parol	propn
Parrol	propn
Rbl	propn
Rbl	propn
Paroli	propn
Parolil	propn
Juhasoni	propn
Parrol	propn
Parol	propn
Juhansonil	propn
Parrol	propn
Juhanson	propn
Parrol	propn
Juhansoni	propn
Juhansonil	propn
Parrol	propn
Juhansonile	propn
Rbl	propn
Parol	propn
Kristjan	propn
Miller	propn
Laatrest	propn
Jaak	propn
Roosenfeldi	propn
Jaak	propn
Rosenfeldt	propn
Kaupmees	propn
Petersoni	propn
Millerile	propn
Jaak	propn
Roosenfeldt	propn
Jaak	propn
Kiis	propn
Paroli	propn
Parrol	propn
Ollev	propn
Ollev	propn
Parol	propn
Jaan	propn
Kiis	propn
Tilga	propn
Parrol	propn
Kristjaan	propn
Hahn	propn
Kristjani	propn
valla	propn
Kristjan	propn
Hahn	propn
Paroli	propn
Parol	propn
Parroli	propn
Parrolil	propn
Parroli	propn
Parroli	propn
Parolil	propn
valla	propn
Parroli	propn
Kristjaan	propn
Hahn	propn
Parroli	propn
Parrol	propn
Kristan	propn
Kristjani	propn
Parrol

kuikatsist	propn
Johann	propn
Johannson	propn
Parroli	propn
Parrolile	propn
Parroli	propn
kuikatsist	propn
Parroli	propn
Parrol	propn
Parrol	propn
Peter	propn
Pilk	propn
Simmon	propn
Oder	propn
Peter	propn
Pilk	propn
Johann	propn
Johannson	propn
Johannson	propn
Parroli	propn
Johanson	propn
Johansoni	propn
Parrol	propn
Johannsoni	propn
Parrol	propn
Johannson	propn
Parrolile	propn
Johannson	propn
Johanson	propn
Parroli	propn
Johannson	propn
Johhannson	propn
Parroli	propn
Johannsonil	propn
Parrol	propn
Johannson	propn
Johannson	propn
Simmon	propn
Odder	propn
Peter	propn
Pilk	propn
Johann	propn
Johannson	propn
Parroli	propn
Parrolile	propn
Parroli	propn
Cop	propn
Parroli	propn
Pornusse	propn
Rbl	propn
Cop	propn
Parrol	propn
kuikatsi	propn
Rbl	propn
Cop	propn
Parrol	propn
Rbl	propn
Cop	propn
Parrol	propn
Rbl	propn
Cop	propn
Parrol	propn
Rbl	propn
Cop	propn
Parrol	propn
Rbl	propn
Cop	propn
Rbl	propn
Cop	propn
Parroli	propn
Rbl	propn
Cop	propn
Parrol	propn
Parrol	propn
Rbl	propn
Parrol	propn


Jaan	propn
Rämmaga	propn
Rijan	propn
Jaan	propn
Rämma	propn
Johann	propn
Saarva	propn
Rbl	propn
Peter	propn
Soodla	propn
Rbl	propn
Hans	propn
Grünfeldt	propn
Rbl	propn
Andres	propn
Reidolf	propn
Rbl	propn
Kop	propn
Villem	propn
Teska	propn
Rbl	propn
Villem	propn
Teska	propn
Jaan	propn
Rämma	propn
Rbl	propn
Rbl	propn
Rbl	propn
Rijast	propn
Rbl	propn
Jaan	propn
Rämma	propn
Jan	propn
Rämma	propn
Johann	propn
Grünfeldt	propn
Andres	propn
Reidolfi	propn
Johann	propn
Grünfeldt	propn
Johann	propn
Grünfeldt	propn
Johann	propn
Grünfeldti	propn
Andres	propn
Reidolf	propn
Rbl	propn
Villem	propn
Teska	propn
Rijan	propn
Valgast	propn
Andress	propn
Reidolf	propn
Jaan	propn
Rämmal	propn
Jaan	propn
Rämma	propn
Johann	propn
Grünfeldt	propn
Jaan	propn
Rämman	propn
Johann	propn
Grünfeldt	propn
Jaan	propn
Rämma	propn
Kirrepist	propn
Johann	propn
Grünfeldt	propn
Jaan	propn
Rämmat	propn
Röngo	propn
Preestri	propn
Schorochovi	propn
Peep	propn
Kirrepist	propn
Preestri	propn
Kerrikoplatsi	propn
Peep	propn
Kuld

Karro	propn
Jaan	propn
Karro	propn
Jaan	propn
Reidolfiga	propn
Jaan	propn
Karro	propn
Märt	propn
Luik	propn
Jaan	propn
Karro	propn
Jaan	propn
Reidolfi	propn
Andres	propn
Must	propn
Paakohtomeess	propn
Märt	propn
Luik	propn
Jaan	propn
Karro	propn
Jaan	propn
Reidolfile	propn
Cop	propn
Jaan	propn
Reidolfi	propn
Cop	propn
Jaan	propn
Reidolfi	propn
Jaan	propn
Reidolfile	propn
Jaan	propn
Karro	propn
Jaan	propn
Karro	propn
Keiserlikko	propn
Jaan	propn
Reidolf	propn
Jaan	propn
Karro	propn
Rbl	propn
Cop	propn
Jaan	propn
Karro	propn
Jaan	propn
Jarro	propn
Jaan	propn
Reidolfi	propn
Jaan	propn
Reidolf	propn
Jaan	propn
Karrol	propn
Haan	propn
Karro	propn
Rbl	propn
Cop	propn
Jaan	propn
Reidolfi	propn
Jaan	propn
Reidolfile	propn
Jaan	propn
Karrol	propn
Jaan	propn
Reidolfi	propn
Jaan	propn
Reidolfi	propn
Jaan	propn
Karro	propn
Keiserlikko	propn
Kihhelkonna	propn
Jaan	propn
Karro	propn
Jaan	propn
Reidolfi	propn
Jaan	propn
Reidolf	propn
Jaan	propn
Reidolf	propn
Jaan	propn
Karro	propn
Möistetu	propn
Jaan

In [9]:
with open('uus_tsv_gazetter.txt', 'w+', encoding='UTF-8') as file:
    for name in set_of_names:
        file.write(name + "\t" + 'propn' + '\n')